# Predictions

In [ ]:
import T03
import time
import math

current_day = 195

df , today_day = T03.get_data(current_day)

next4days = T03.make_future_predictions(today_day)
# print('-'*10)
# print(next4days.to_string(index=False))
# print('-'*10)

cash_df = T03.get_cash_data(today_day)
stinfo ,working_df = T03.get_stations_info(today_day)

pending_jobs = max(0,working_df['JOBIN'].sum()  -  working_df['JOBOUT'].sum())
pending_jobs_f = pending_jobs > 7

job_rev = working_df.iloc[-1]['JOBREV']
job_rev_f = job_rev < 1100

job_time = working_df.iloc[-1]['JOBT'].mean()
job_time_f = job_time > .5

R ,S , H=  df.iloc[today_day-1]['JOBIN_PRED'],800, 600*.1/365
lead_time , demand_std,service_level = 5 , df.iloc[today_day-50:today_day]['JOBIN'].std(),2.5

eoq = math.ceil(math.sqrt(  2 *  R * S / H)) 
cycle_stock = df.iloc[today_day:today_day+5]['JOBIN_PRED'].sum()
safety_stock =   math.ceil ( service_level * (lead_time**.5) * demand_std )
rop = cycle_stock + safety_stock  

# print(eoq,rop,cycle_stock,safety_stock)
days_remaining = 268 - today_day
inventory_needed = df.iloc[150:today_day]['JOBIN'].mean() * days_remaining
inventory_std = df.iloc[150:today_day]['JOBIN'].std() / math.sqrt(days_remaining)
inventory_needed_u = 12.75 * days_remaining


current_inv = df.iloc[today_day-1]['INV']/60

## OLS

In [ ]:
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm

df50 = df.head(50)
y = df50.JOBIN
X = sm.add_constant(df50.day)

model = sm.OLS(y, X)
model.fit()
results = model.fit()
print(results.summary())

# Today is  Day {{today_day}}


| Check   |      Value      | Warning Flag |
|----------|-------------:|------:|
| Pending Jobs |  {{pending_jobs}} | {{pending_jobs_f}} |
| Job Revenue |    $ {{job_rev}}   |   {{job_rev_f}} |
| Job Lead Time |    {{job_time}} day  |   {{job_time_f}} |

Suggested : EOQ **{{eoq}}** , ROP **{{100}}**

**pending** Jobs are a spillover from yesterday. A number close to 0.3 times **Station Capacity** is deemed unhealthy 


In [ ]:
T03.print_summary()

In [ ]:
%matplotlib inline
import T03
import matplotlib.pyplot as plt
from pylab import rcParams
import numpy as np

rcParams['figure.figsize'] = 16, 12
rcParams['lines.linewidth'] = 2

perf_df = df.iloc[:today_day].copy()


T03.plotx(plt,1,perf_df,['JOBIN','JOBOUT'],(3,28),"Orders per day -10 days - now")
T03.plotx(plt,2,perf_df,['S1UTIL','S2UTIL','S3UTIL'],(-.1,1.2),"Utilizations -10 days - now")
T03.plotx(plt,3,perf_df,['S1Q','S2Q','S3Q'],(-1,50),"Queues -10 days - now")

T03.plotx(plt,4,perf_df,['JOBT',],(.2,1.5),"Lead Time -10 days - now")
T03.plotx(plt,5,perf_df,['JOBREV'],(-1,1500),"Revenue Per Order -10 days - now")

T03.plotx(plt,6,cash_df,['JOBIN','JOBOUT','JOBIN_PRED'],(0,30),"Orders",today_day)


# print (invdf['INV'])
T03.plotx(plt,7,cash_df,['INV'],(-1,900),"Inventory",today_day)

T03.plotx(plt,8,cash_df,['CASH','CUM_EARN'],(6,3000),"CASH Predictions",today_day)
# T03.plotx(plt,8,cash_df,['CASH','CUM_EARN','CASH_1K',"CASH_750"],(6,3000),"CASH Predictions",today_day)

# print (cash_df.iloc[today_day-2:today_day+2][['day','CASH','INV']])

# Autopilot Action Log

In [ ]:
with open("/home/ajar/work/little/R01.log",'rb') as f:
    for line in f:
        print(line)

### Group Standings

In [ ]:
import pandas as pd        
import time


group_df = T03.get_current_assets()

group_df = group_df

group_df['g4-g2'] =  group_df['g4'] - group_df['g2']
# T03.plottime(plt,11,group_df,['g4-g2'],(0,50000),"G4-G2")
for i in [1,2,3,5]: group_df['g4-%d'%i] =   group_df['g%d'%i] - group_df['g4'] 
T03.plottime(plt,14,group_df,['g4-1','g4-2','g4-3','g4-5'],(-35000,35000),"G4-Deltas")


# T03.plottime(plt,10,group_df,['g%dass'%i for i in range(1,6)],(1.6e6,1.95e6),"Current Assets")
T03.plottime(plt,13,group_df,['g%d'%i for i in range(1,6)],(1.2e6,2.263e6),"Current Cash")
T03.plottime(plt,12,group_df,['g%dinv'%i for i in range(1,6)],(10,800),"Inventory")
# T03.plottime(plt,14,group_df,['g%ddelta'%i for i in range(1,6)],(1,5000),"Deltas")
# T03.plottime(plt,15,group_df,['g4ass'],(1.8e6,1.96e6),"Our Assets")


# print(group_df.tail()[['time','day']+['g%ddelta'%i for i in range(1,6)]])
# print(group_df.tail()[['time','day']+['g%d'%i for i in range(1,6)]])
# print(group_df.tail()[['time','day']+['g%dass'%i for i in range(1,6)]])
# print(group_df[['time','day']+['g%dinv'%i for i in range(1,6)]])
# print(group_df.tail()[['time','day']+['g4-1','g4-2','g4-3','g4-5']])
# xdf = df.iloc[today_day-20:today_day].copy()
# xdf['JOBIN'] = xdf['JOBIN'].cumsum() 
# print(xdf[['day','JOBIN']])

### Past 10 day order mixes

In [ ]:
print(df.iloc[today_day-10:today_day][['day','JOBIN','JOBOUT_075', 'JOBOUT_100', 'JOBOUT_125', 'JOBT_100', 'JOBT_125']].to_string(index=False))

# df.iloc[150:190][['JOBIN']].plot(kind='hist', alpha=0.5)